In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\mayan\\Videos\\Projects\\Medical-Chatbot-GenAI\\research'

In [3]:
import os

os.chdir("../")     # chdir means change directory

In [4]:
%pwd

'c:\\Users\\mayan\\Videos\\Projects\\Medical-Chatbot-GenAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract Data from PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
# extracted_data

In [9]:
# Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=25)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 4896


In [11]:
# text_chunks

In [20]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [26]:
# Download the Embeddings from google genai

def download_google_genai_embeddings():
    embeddings=GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    return embeddings

In [27]:

embeddings = download_google_genai_embeddings()

In [28]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 768


In [30]:
# query_result

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [34]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=768, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [39]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGlE_API_KEY"] = GOOGLE_API_KEY

In [40]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [41]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [42]:
docsearch

In [43]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [44]:
retrieved_docs = retriever.invoke("What is Acne?")

In [45]:
retrieved_docs

[Document(id='39e10d8a-8465-4188-99ab-d691e2808315', metadata={'page': 239.0, 'page_label': '240', 'source': 'Data\\Medical_book.pdf'}, page_content='Normally, sebum flows out onto the skin and hair to\nform a protective coating, but when it cannot get out,\nsmall swellings develop on the skin surface. Bacteria\nand dead skin cells can also collect that can cause\ninflammation. Swellings that are small and not\ninflamed are whiteheads or blackheads. When they\nbecome inflamed, they turn into pimples. Pimples that\nfill with pus are called pustules.\nAcne cannot be cured, but acne drugs can help clear\nthe skin. Benzoyl peroxide and tretinoin work by mildly\nirritating the skin. This encourages skin cells to slough'),
 Document(id='dd3722c5-caac-47fd-bc2f-b969fd1f4169', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\Medical_book.pdf'}, page_content='• Stress. Emotional stress may contribute to acne.\nAcne is usually not conspicuous, although inflamed\nlesions may cause pai

In [46]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.4, max_tokens=500)

In [47]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [48]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [52]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne is a skin condition characterized by raised bumps, pimples, and cysts forming on the face, neck, shoulders, and upper back.  It's caused by bacteria and can be emotionally distressing due to cosmetic concerns and potential scarring.  Treatment may involve a physician or dermatologist.


In [51]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I am sorry, but this document does not contain the answer to this question.  I do not have the ability to access external websites or specific files online.
